In [1]:
from PIL import Image
import numpy as np
import glob
import os
import pandas as pd

In [2]:
images_path = "image_train/"
texts_path = "txt_train/"
sentence_file_path = "sentences.csv"
images_output_path = "image_output/"
images = glob.glob(images_path + "*.jpg")
texts = glob.glob(texts_path + "*.txt")

In [3]:
def preprocessing_image(image, position_list):
    # 将原始坐标数组转化为点坐标数组
    def pos2points(position_list):
        res = []
        tmp = []
        for i in range(len(position_list)):
            tmp.append(position_list[i])
            if(i % 2 == 1):
                tmp = np.array(tmp)
                res.append(tmp)
                tmp = []
        return res
    pts = pos2points(position_list)
    # 求矩形高
    h = int(np.linalg.norm(pts[0]-pts[1]))
    # 求矩形宽
    w = int(np.linalg.norm(pts[1]-pts[2]))
    # 进行变换，旋转，水平翻转
    res_img = image.transform((w,h),Image.QUAD,tuple(position_list)).rotate(180)
    # 如果是左右倒了
    if pts[0][1]-pts[1][1] < 0:
        res_img = res_img.transpose(Image.FLIP_TOP_BOTTOM)
    # 如果是上下倒了
    if pts[0][0] - pts[2][0] < 0:
        res_img = res_img.transpose(Image.FLIP_LEFT_RIGHT)
    return res_img

In [4]:
index = 0
f = open(sentence_file_path, "w", encoding='utf-8')
f.write("index,content\n")
for i in range(len(images)):
    text_content = pd.read_csv(texts[i], header=None,error_bad_lines=False,quoting=3, encoding='utf-8').loc[:,0:8]
    text_content = text_content[text_content[8] != "###"]
    image_content = Image.open(images[i]).convert('RGB')
    if i % 100 == 0:
        print(i,index)
    for x in text_content.itertuples():
        try:
            a = preprocessing_image(image_content, list(x[1:9]))
        except ZeroDivisionError:
            pass
        a.save(images_output_path + str(index) + ".jpg", "jpeg")
        f.write(str(index) + ',' + str(x[9]) + '\n')
        index = index + 1
f.close()

0 0


b'Skipping line 12: expected 9 fields, saw 12\nSkipping line 13: expected 9 fields, saw 10\n'
b'Skipping line 3: expected 9 fields, saw 10\n'
b'Skipping line 4: expected 9 fields, saw 10\nSkipping line 14: expected 9 fields, saw 10\n'


100 1305


b'Skipping line 12: expected 9 fields, saw 10\nSkipping line 15: expected 9 fields, saw 10\nSkipping line 17: expected 9 fields, saw 11\n'
b'Skipping line 5: expected 9 fields, saw 10\nSkipping line 13: expected 9 fields, saw 10\n'


200 2617
300 4102


...

b'Skipping line 8: expected 9 fields, saw 10\n'
b'Skipping line 6: expected 9 fields, saw 10\n'


9900 141873


b'Skipping line 51: expected 9 fields, saw 10\n'
b'Skipping line 8: expected 9 fields, saw 11\nSkipping line 13: expected 9 fields, saw 11\n'


10000 143312


b'Skipping line 11: expected 9 fields, saw 10\n'
b'Skipping line 9: expected 9 fields, saw 10\nSkipping line 13: expected 9 fields, saw 10\n'
b'Skipping line 4: expected 9 fields, saw 10\n'
